# Rolf Fourier functions in python
greektype=1 Price,
greektype=2 delta,
greektype=3 Gamma,
greektype=4 Vega

In [5]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import brentq as root
import scipy.integrate as integrate

In [6]:
def HestonFourier(spot,timetoexp,strike,r,divyield,V,theta,kappa,sigma,rho,greek=1):
    X = np.log(spot/strike)+(r-divyield)*timetoexp
    kappahat = kappa-0.5*rho*sigma
    xiDummy = kappahat**2+0.25*sigma**2
    
    def integrand(k):
        xi = np.sqrt(k**2*sigma**2*(1-rho**2)+2j*k*sigma*rho*kappahat+xiDummy)
        PsiP = -(1j*k*rho*sigma+kappahat)+xi
        PsiM = (1j*k*rho*sigma+kappahat)+xi
        alpha = -kappa*theta*(PsiP*timetoexp + 2*np.log((PsiM + PsiP*np.exp(-xi*timetoexp))/(2*xi)))/sigma**2
        beta = -(1-np.exp(-xi*timetoexp))/(PsiM + PsiP*np.exp(-xi*timetoexp))
        numerator = np.exp((-1j*k+0.5)*X+alpha+(k**2+0.25)*beta*V)

        if(greek==1):
            dummy = (numerator/(k**2+0.25)).real
        if(greek==2):
            dummy = ((0.5-1j*k)*numerator/(spot*(k**2+0.25))).real
        if(greek==3):
            dummy = -(numerator/spot**2).real
        if(greek==4):
            dummy = (numerator*beta).real
    
        return dummy

    dummy=integrate.quad(integrand,-100,100)[0]

    if (greek==1):
        dummy = np.exp(-divyield*timetoexp)*spot-strike*np.exp(-r*timetoexp)*dummy/(2*np.pi)

    if(greek==2):
        dummy = np.exp(-divyield*timetoexp)-strike*np.exp(-r*timetoexp)*dummy/(2*np.pi)

    if(greek==3):
        dummy = -strike*np.exp(-r*timetoexp)*dummy/(2*np.pi)
  
    if(greek==4):
        dummy = -strike*np.exp(-r*timetoexp)*dummy/(2*np.pi)

    return dummy 

In [7]:
#function used to calculate the L(t,v) process
def L(t,v):
    return (v-theta)/kappa*(1-np.exp(-kappa*(T-t)))+theta*(T-t)

#the derivative of L wrt. v
def dL(t):
    return (1-np.exp(-kappa*(T-t)))/kappa

#delta_t^1
def delta1(s,v,t):
    timetoexp = T-t
    return HestonFourier(spot=s, timetoexp=timetoexp, strike=K, r=r, divyield=0, V=v, theta=theta, kappa=kappa, sigma=sigma, rho=rho, greek=2)

vdelta1 = np.vectorize(delta1)
#delta_t^2
def delta2(s,v,t):
    timetoexp = T-t
    return HestonFourier(spot=s, timetoexp=timetoexp, strike=K, r=r, divyield=0, V=v, theta=theta, kappa=kappa, sigma=sigma, rho=rho, greek=4)/dL(t)

vdelta2 = np.vectorize(delta2)